# oneTBB Concurrent Containers

##### Sections
- [oneTBB Concurrent Containers](#oneTBB-Concurrent-Containers)
- _Code_: [A Producer-Consumer Application with tbb::concurrent_queue](#A-Producer-Consumer-Application-with-tbb::concurrent_queue)

## Learning Objectives
* Learn how thread-unsafe uses of a standard container might be addressed by using a oneTBB concurrent container

# oneTBB Concurrent Containers

The oneTBB library provides a number of 
[concurrent containers](https://spec.oneapi.com/versions/latest/elements/oneTBB/source/containers.html),
including `concurrent_vector`, `concurrent_queue`, `concurrent_bounded_queue`, `concurrent_priority_queue`,
`concurrent_hash_map`, `concurrent_unordered_map` and more. These classes provide optimized containers that 
permit multiple threads to simultaneously invoke certain functions on the same container.

## A Producer-Consumer Application with tbb::concurrent_priority_queue

In this section, we will implement a very simple producer-consumer application that uses a shared 
priority queue. A producer thread will generate 1000 items, putting them into a queue and a consumer 
thread will read the items from the queue, generating a sum of the values as a result.

### Run the sequential baseline implementation

In our baseline implementation, the producer thread and the consumer thread are the same thread, the main thread.
There is no concurrency expressed in our serial code, but if there was, we would have a race condition since we
use a `std::priority_queue`. In the `while`-loop, we call `empty` before calling `pop`. If more than one thread
was simultaneously popping from this shared queue, the queue might appear non-empty at the test in the `while`-loop
but be empty by the time `pop` is called. This problem does not exist in our serial code since only one thread is
used. In the next section, we will add concurrency and the potential for such a race condition.

In this section, just inspect the sequential code below - there are no modifications necessary. Run the first cell 
to create the file, then run the cell below it to compile and execute the code. This represents the baseline for 
our producer-consumer exercise.

1. Inspect the code cell below, then click run ▶ to save the code to a file
2. Run ▶ the cell in the __Build and Run the baseline__ section below the code snippet to compile and execute the code in the saved file

In [ ]:
%%writefile lab/q-serial.cpp
//==============================================================
// Copyright (c) 2020 Intel Corporation
//
// SPDX-License-Identifier: Apache-2.0
// =============================================================

#include <iostream>
#include <queue>

int main() {
  int sum (0);
  int item;

  std::priority_queue<int> myPQ;
 
  for(int i=0; i<10001; i+=1) {
    myPQ.push(i);
  }
 
  while( !myPQ.empty() ) {
    sum += myPQ.top();
    myPQ.pop();
  }

  // prints "total: 50005000" (for 0,10001,1)
  std::cout << "total: " << sum << '\n';
  return 0;
}

### Build and Run the baseline
Select the cell below and click Run ▶ to compile and execute the code above:

In [ ]:
! chmod 755 q; chmod 755 ./scripts/run_q-serial.sh; if [ -x "$(command -v qsub)" ]; then ./q scripts/run_q-serial.sh; else ./scripts/run_q-serial.sh; fi

### Implement a parallel version with tbb::concurrent_priority_queue

In this section, we modify the example to create one producer thread
and two consumer threads that run concurrently. To eliminate the potential
race on the call to `empty` and `pop`, we replace the `std::priority_queue` 
with `tbb::concurrent_priority_queue`. In fact, if you run the code below before
making the required modifications, the output might include a segfault.

The key interfaces in `tbb::concurrent_priority_queue` needed for this exercise 
are reproduced below:

```cpp
namespace tbb {
  template <typename T, typename Compare = std::less<T>,
            typename Allocator = cache_aligned_allocator<T>>
  class concurrent_priority_queue {
  public:
    concurrent_priority_queue();
    void push( const value_type& value );
    bool try_pop( value_type& value );
  };
```

You can find detailed documentation for ``concurrent_queue`` [here](https://spec.oneapi.com/versions/latest/elements/oneTBB/source/containers/concurrent_priority_queue_cls.html).

For this exercise, complete the following steps:

1. Inspect the code cell below and make the following modifications.
  1. Replace the type `std::priority_queue` with `tbb::concurrent_priority_queue`.
  2. Replace the invocation of `empty`, `top` and `pop` with a single call to `try_pop`.
2. When the modifications are complete, click run ▶ to save the code to a file. 
3. Run ▶ the cell in the __Build and Run the modified code__ section below the code snippet to compile and execute the code in the saved file.

In [ ]:
%%writefile lab/q-parallel.cpp
//==============================================================
// Copyright (c) 2020 Intel Corporation
//
// SPDX-License-Identifier: Apache-2.0
// =============================================================

#include <iostream>
#include <queue>
#include <thread>
#include <tbb/tbb.h>

#define INCORRECT_QUEUE_TYPE std::priority_queue<int>;

// STEP A: Replace std::priority_queue with tbb::concurrent_priority_queue
using queue_type = INCORRECT_QUEUE_TYPE;
        
int consume(queue_type &myPQ) {
  int local_sum = 0;
  int value = -1;
    
  bool consuming = true;
  while(consuming) {
    // STEP B: replace the pattern of empty, top and pop 
    //         with a single call to myPQ.try_pop(value)
    if (!myPQ.empty()) {
      value = myPQ.top();
      myPQ.pop();
      if (value == -1)
        consuming = false;
      else
        local_sum += value;
    }
  }
  return local_sum;
}

int main() {
  int sum (0);
  int item;

  queue_type myPQ;
 
  std::thread producer([&]() {
    for(int i=0; i<10001; i+=1) {
      myPQ.push(i);
    }
    // to signal the end to the two consumers
    myPQ.push(-1);
    myPQ.push(-1);
  });

  int local_sum1 = 0, local_sum2 = 0;
  std::thread consumer1([&]() { local_sum1 = consume(myPQ); });
  std::thread consumer2([&]() { local_sum2 = consume(myPQ); });

  producer.join();
  consumer1.join();
  consumer2.join();
    
  // prints "total: 50005000" (for 0,10001,1)
  std::cout << "total: " << local_sum1 + local_sum2 << '\n';
  return 0;
}

### Build and Run the modified code

Select the cell below and click Run ▶ to compile and execute the code that you modified above:

In [ ]:
! chmod 755 q; chmod 755 ./scripts/run_q-parallel.sh; if [ -x "$(command -v qsub)" ]; then ./q scripts/run_q-parallel.sh; else ./scripts/run_q-parallel.sh; fi

## Producer-Consumer Solution (Don't peak unless you have to)

In [ ]:
%%writefile solutions/q-parallel.cpp
//==============================================================
// Copyright (c) 2020 Intel Corporation
//
// SPDX-License-Identifier: Apache-2.0
// =============================================================

#include <iostream>
#include <queue>
#include <thread>
#include <tbb/tbb.h>

// STEP A: Replace std::priority_queue with tbb::concurrent_priority_queue
using queue_type = tbb::concurrent_priority_queue<int>;
        
int consume(queue_type &myPQ) {
  int local_sum = 0;
  int value = -1;
    
  bool consuming = true;
  while(consuming) {
    // STEP B: replace the pattern of empty, top and pop 
    //         with a single call to myPQ.try_pop(value)
    if (myPQ.try_pop(value)) {
      if (value == -1)
        consuming = false;
      else
        local_sum += value;
    }
  }
  return local_sum;
}

int main() {
  int sum (0);
  int item;

  queue_type myPQ;
 
  std::thread producer([&]() {
    for(int i=0; i<10001; i+=1) {
      myPQ.push(i);
    }
    // to signal the end to the two consumers
    myPQ.push(-1);
    myPQ.push(-1);
  });

  int local_sum1 = 0, local_sum2 = 0;
  std::thread consumer1([&]() { local_sum1 = consume(myPQ); });
  std::thread consumer2([&]() { local_sum2 = consume(myPQ); });

  producer.join();
  consumer1.join();
  consumer2.join();
    
  // prints "total: 50005000" (for 0,10001,1)
  std::cout << "total: " << local_sum1 + local_sum2 << '\n';
  return 0;
}

In [ ]:
! chmod 755 q; chmod 755 ./scripts/run_q-solution.sh; if [ -x "$(command -v qsub)" ]; then ./q scripts/run_q-solution.sh; else ./scripts/run_q-solution.sh; fi